In [ ]:
import identification_py2 as ob
import matplotlib.pyplot as plt
import glob #for returning files having the specified path extension
import os #checking for empty file
import pathlib #for creating new path
%pylab inline

###### Passing all the data into arrays 

In [ ]:
task_null           = sorted(glob.glob('step_log_new/*/*task*.log')) #corresponds to .log files that has data related to the first position
control_null        = sorted(glob.glob('step_log_new/*/*control*.log'))
task_remaining      = sorted(glob.glob('step_log_new/*/*task*.log.*')) #corresponds to remaining log.'n' files
control_remaining   = sorted(glob.glob('step_log_new/*/*control*.log.*'))
task_v              = sorted(task_null + task_remaining) #set of all task_velocity logs
control_o           = sorted(control_null + control_remaining) #set of all control logs
observations        = len(task_null) #total number of experiments conducted/observations taken
positions           = int(len(task_v) / observations) #number of points in the given task space
task_full           = [] #A task_velocity list whose each element is a list of similar log files i.e from the same position
control_full        = [] #A control_output list whose each element is a list of similar log files i.e from the same position

for i in range(0, positions):
    task_full.append([])
    control_full.append([])
    for j in range(0, observations):
        task_full[i].append(task_v[i + (j * positions)])
        control_full[i].append(control_o[i + (j * positions)])

count = 0 #counter that returns the number of empty files
for i in range(0, positions):
    for j in range(0, observations):
        if os.stat(task_full[i][j]).st_size == 0:
            count = count + 1
            
for i in range(0, positions):
    for j in range(0, observations-count):
        if os.stat(task_full[i][j]).st_size == 0:
            del(task_full[i][j])
            del(control_full[i][j])
            
# Reading all the data into a dataframe array
df_ist_soll = []
for i in range(0, positions):
    df_ist_soll.append([])
    for j in range(0, observations):
        try:
            df_ist_soll[i].append(ob.batch_read_data(control_full[i][j], task_full[i][j]))
        except:
            continue

###### Displaying all the observations 

In [ ]:
# The first try except code avoids the errors arising due to the already existing Overview directory. 
# The second try except code avoids the errors resulting from the plotting of the empty data file 
try:
    os.makedirs('Overview/')
except OSError, e:
    if e.errno != os.errno.EEXIST:
        raise   
    pass

for i in range(0, positions):
    fig = plt.figure(figsize = (10,30))
    fig.suptitle('Position %s'%(i + 1), fontsize = 20, fontweight = 'bold')
    for j in range(0, observations): 
        try:
            ax = fig.add_subplot(observations, 1, j + 1)
            ax.set_title('Observation %s'%(j + 1))
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            plt.plot(df_ist_soll[i][j]) 
        except:
            pass
    plt.savefig('Overview/Position %s.png'%(i + 1))